In [1]:
import numpy as np
%matplotlib inline
from pylab import *
# from numpy import linalg
# import math
# import cProfile
import time

# Introduction

The goal of this project is to solve a system of linear equations using Cramer's rule instead of using the current standard methods.

We will consider linear equations in the form of $Ax=b$, where $A$ is a non-singular $n\times n$ matrix, $b$ is a $n\times 1$ unknown vector, and $b$ is a given $n\times 1$ vector. Cramer's rules states that
\begin{align*}
	x_i = \frac{\det(A_i(b))}{\det(A)}, i=1,2,\cdots,n
\end{align*}
where $A_i(b)$ denotes the matrix $A$ with its ith column replaced by $b$.

The standard method of computing the determinant by recursion of minors is not an efficient computation algorithm. In fact, its complexity is of $O(N!)$, which makes it useless for real world problems. Instead, methods are used to row reduce the matrix to obtain upper and lower triangular matrices and then compute the determinate from those.

## Example

Consider the following system of equations:
\begin{align*}
	a_{1,1}x_1 + a_{1,2}x_2 + a_{1,3}x_3 = b_1 \\
	a_{2,1}x_1 + a_{2,2}x_2 + a_{2,3}x_3 = b_2 \\
	a_{3,1}x_1 + a_{3,2}x_2 + a_{3,3}x_3 = b_3
\end{align*}
With
\begin{align*}
	&A = \begin{bmatrix}
		a_{1,1} & a_{1,2} & a_{1,3} \\
		a_{2,1} & a_{2,2} & a_{2,3} \\
		a_{3,1} & a_{3,2} & a_{3,3}
	\end{bmatrix}, &
	&x = \begin{bmatrix}x_1 \\	x_2 \\	x_3	\end{bmatrix}, & 
	b = \begin{bmatrix} b_1 \\	b_2 \\	b_3	\end{bmatrix}&
\end{align*}
Then
\begin{align*}
	x_1 &= \frac{\det\begin{bmatrix}
			b_1 & a_{1,2} & a_{1,3} \\
			b_2 & a_{2,2} & a_{2,3} \\
			b_3 & a_{3,2} & a_{3,3}
		\end{bmatrix}
	}{\det(A)}, &
	x_2 = \frac{\det\begin{bmatrix}
			a_{1,1} & b_1 & a_{1,3} \\
			a_{2,1} & b_2 & a_{2,3} \\
			a_{3,1} & b_3 & a_{3,3}
		\end{bmatrix}
	}{\det(A)}, & &
	x_3 = \frac{\det\begin{bmatrix}
			a_{1,1} & a_{1,2} & b_1 \\
			a_{2,1} & a_{2,2} & b_2 \\
			a_{3,1} & a_{3,2} & b_3
		\end{bmatrix}
	}{\det(A)}&
\end{align*}

# Current Approaches

Current technology with python to solve a system of linear equations uses *scipy.linalg.solve*. This in turn uses routines from LAPACK, "The generic, symmetric, Hermitian and positive definite solutions are obtained via calling ?GESV, ?SYSV, ?HESV, and ?POSV routines of LAPACK respectively."

There are specialized solutions for known special matrices, but we are most interested in the generic solver. From the documentation of ?GESV, we have, 

> ?GESV computes the solution to a real system of linear equations
> \begin{align*} 	A * X = B \end{align*}
> where A is an N-by-N matrix and X and B are N-by-NRHS matrices.
>
> The LU decomposition with partial pivoting and row interchanges is used to factor A as
> \begin{align*}A = P * L * U\end{align*}
> where P is a permutation matrix, L is unit lower triangular, and U is upper triangular.  The factored form of A is then used to solve the system of equations $A * X = B$.

The complexity of the ?GESV algorithm is indicated as $O(.67N^3)$. This is obtained from table 3.13 in https://www.netlib.org/lapack/lug/node71.html.

## Determinant

Determinants of triangular matrices are easily calculated as the product of the diagonal entries. Thus $\det(L)$ and $\det(U)$ are easily calculated by $O(n)$ operations. Since P is a permutation matrix, its determinant is trivial to calculate using the method of minors and is of order $O(n)$. $det(P)$ equals either $1$ or $-1$. So that we have:
\begin{align*}
	\det(A) = \det(P) * \det(L) * \det(U)
\end{align*}
This is the method used by both numpy and scipy to calculate a determinant.

## Cramer's Rule and LU Decomposition

Based on the above discussion, Cramer's Rule could be used to solve systems of equations by calculating the various determinants using LU decomposition as an intermediate step. This would give us computation complexity on the order of $O(.67N^3)$ for LU Decomposition $\times$ $O(n)$ for the number of variables to solve. With the result of at best $O(.67N^4)$. This is due to the need to perform LU Decomposition for each $A_i(b)$. If there were a method to reduce this need we might have a competitive algorithm computationally.

I did find one approach that does claim to reduce this complexity to about $O(N^3)$. This is from the Journal of Discrete Algorithms, http://web.eecs.utk.edu/~ielhanan/Papers/JDA2011.pdf, and it uses Chio's condensation method and a few other tricks. I am not certain of this algorithm yet.

# Test Matrices

Test matrices are an important part of validating algorithms, both for computation speed and accuracy. There are many matrices available for direct download from https://math.nist.gov/MatrixMarket/index.html. These may be downloaded in MatrixMarket format and then pulled into the test program with the following code snippet.

In [2]:
from scipy import io
print(io.mminfo('rbs480a.mtx'))
a = io.mmread('rbs480a.mtx')
A=  a.todense()
n = len(A)

(480, 480, 17088, 'coordinate', 'real', 'general')


However, many test matrices are generated. Often, these were generated using The Matrix Computation Toolkit for Matlab. A very useful python package to generate a variety of test matrices is in Scipy.sparse. See the code example:

In [3]:
# DOCUMENTATION: https://docs.scipy.org/doc/scipy/reference/sparse.html
# https://docs.scipy.org/doc/scipy/reference/stats.html
import scipy.sparse as sparse
import scipy.stats as stats

In [4]:
# sets reproducibility
np.random.seed(5) 
# Poisson
rvs = stats.poisson(18, loc=10).rvs
# generate sparse 5x5 random matrix with density 0.75
A = sparse.random(5, 5, density=0.75, data_rvs=rvs)
rvs = stats.norm(loc=10, scale=20).rvs
# create sparse 3x3 matrix with density 0.5
B = sparse.random(3, 3, density=0.50, data_rvs=rvs)
A = A.toarray()
B = B.todense()
A, B

(array([[20., 29., 29.,  0., 29.],
        [23.,  0., 26.,  0., 30.],
        [28., 30., 30., 29., 31.],
        [ 0., 23., 25., 34., 25.],
        [29.,  0.,  0., 24., 26.]]),
 matrix([[ 0.        ,  0.        , 12.49613651],
         [ 0.        ,  0.        ,  0.        ],
         [23.13238941, 55.20213547,  7.8567203 ]]))

Is there a difference in using either of these forms, matrix or array? I think that either will work fine but numpy is most suitable for working with arrays for slicing and dicing. So mostly, this work will use the array form of the matrix.

# Solve a linear system by Cramer's formula
Consider the **linear systems** in the matrix form $Ax = b$

## A Function for Cramer's rule

In [5]:
def cramer_solve(A, b):
    """Solve a system of linear equations using Cramer's Rule and LU Decomposition.
    Returns the solution vector."""
    
    # LU Decomposition
    p,l,u = sla.lu(A)
    DA = sla.det(p)* sla.det(l)*sla.det(u)
    
    # Solution storage
    x = np.zeros((n,1))
    x.astype(double)
    
    # Loop to find each solution
    for i in range(len(b)):
        Ai = np.copy(A)
#         print(np.shape(Ai), np.shape(b))
        Ai[:,i] = b[:,0]
        p,l,u = sla.lu(Ai)
        DAi = sla.det(p)* dia_det(l)*dia_det(u)
        x[i] = DAi / DA
#         print(x[i])
    return x

def dia_det(A):
    det = 1
    for i in range(len(b)):
        det = det *  A[i,i]
    return det

### Using cramer_solve

In [6]:
# Set up a run and create test matrices
import scipy.sparse as sparse
import scipy.stats as stats
import scipy.linalg as sla

In [7]:
# Create test matrices
# Enter a value of n
n=100
# Sets reproducibility
np.random.seed(3) 
# Use Poisson
rvs = stats.poisson(18, loc=10).rvs
# Generate test matrices A and b
A = sparse.random(n, n, density=0.75, data_rvs=rvs).toarray()
b = sparse.random(n, 1, density=0.9, data_rvs=rvs).toarray()

In [8]:
# cramer_solve
x = cramer_solve(A,b)

# numpy solve
y=linalg.solve(A,b)

print ("Approximate total absolute error compared to built-in solver")
print(sum(abs(x-y)))

Approximate total absolute error compared to built-in solver
2.179978420002726e-12


### Problems

* The determinant calculations can overflow the data type
* Some conditioning will be needed to prevent overflow
* Duplication of some calculations
* This is not an optimized algorithm

# Time to run

In [11]:
# Solve Ax=b by Cramer formulas
t0 = time.time()
# Set iterations and matrix size
i=100
for n in range(10, 10+i):
    np.random.seed(n) 
    rvs = stats.poisson(18, loc=10).rvs
    A = sparse.random(n, n, density=0.75, data_rvs=rvs).toarray()
    b = sparse.random(n, 1, density=0.9, data_rvs=rvs).toarray()
    x = cramer_solve(A,b)
t1 = time.time() 
print('CPU time for Cramer_solve = %g seconds'%(t1 - t0))

CPU time for Cramer_solve = 1.24071 seconds


In [12]:
# Solve Ax=b by built-in methods
t0 = time.time()
# Set iterations and matrix size
i=100
for n in range(10, 10+i):
    np.random.seed(n) 
    rvs = stats.poisson(18, loc=10).rvs
    A = sparse.random(n, n, density=0.75, data_rvs=rvs).toarray()
    b = sparse.random(n, 1, density=0.9, data_rvs=rvs).toarray()
    x = linalg.solve(A,b)
t1 = time.time() 
print('CPU time for Cramer_solve = %g seconds'%(t1 - t0))

CPU time for Cramer_solve = 0.145641 seconds


In [ ]:
# cProfile.run(linalg.solve(A,b))